In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

from functools import partial

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from online_dp import SliceDPModel, LinearModule, MLPModule
from synthetics_utils import generate_multi_mode_data, generate_label_matrix

import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

## Simulating the Simulations

In [2]:
def generate_data(N,accs,covs,op):
    #feature and label generation
    mus = [
    np.array([-3, 0]), # Mode 1: Y = -1
    np.array([3, 0]), # Mode 2: Y = 1
    np.array([6, -3]) # Mode 3: Y = -1
    ]
    props = [0.25, 0.5, 0.25]
    labels = [-1, 1, -1]
    
    X, Y, C = generate_multi_mode_data(N, mus, props, 
                                               labels, plot=False)
    
    #labeling function generation
    L, overlap_idx = generate_label_matrix(N, accs, covs, 
                                           Y, C, overlap_portion=op)
    
    return X,Y,L,overlap_idx

In [3]:
def train_models(X,L):
    m = np.shape(L)[1]
    d = X.shape[1]


    model_0 = SliceDPModel(d, LinearModule, m, accs, r=2, rw=False)
    model_0.train(X, L, batch_size=1000, n_epochs=250, lr=0.1, print_every=250)

    input_module = partial(MLPModule, middle_dims=[4,4])
    model_1 = SliceDPModel(d, input_module, m, accs, r=2, rw=False)
    model_1.train(X, L, batch_size=1000, n_epochs=250, lr=0.1, print_every=250)

    model_2 = SliceDPModel(d, LinearModule, m, accs, r=2, rw=True)
    model_2.train(X, L, batch_size=1000, n_epochs=250, lr=0.1, print_every=250)
    
    return model_0, model_1, model_2

In [4]:
def eval_model(model,X,L,Y,overlap_idx):
    overall_score = model.score(X, Y)
    slice_scores = model.score_on_LF_slices(X, Y, L)
    try:
        overlap_scores = model.score_on_LF_slices(X[overlap_idx,:], Y[overlap_idx], L[overlap_idx,:])
    except:
        overlap_scores = [0,0,0]
    return  [list(slice_scores)+list(overlap_scores)+list([overall_score])][0]

In [5]:
def plot_error(rpca_errs, amc_errs, idx, iters, vals_list):
    """
    Plot a line graph with 95% CI shaded!
    """

    #create pandas dataframe
    m_list = vals_list
    rpca_errs_dataframe = pd.DataFrame([obj[idx] for obj in rpca_errs])
    rpca_errs_dataframe['m'] = pd.Series(m_list*iters, index=rpca_errs_dataframe.index)
    amc_errs_dataframe = pd.DataFrame([obj[idx] for obj in amc_errs])
    amc_errs_dataframe['m'] = pd.Series(m_list*iters, index=amc_errs_dataframe.index)

    #create plot
    sns.lineplot(x='m', y=0, data=rpca_errs_dataframe, marker=True, palette=sns.color_palette("GnBu_d",2))
    sns.lineplot(x='m', y=0, data=amc_errs_dataframe, marker=True, palette = sns.color_palette("GnBu_d",2))
    if idx == 6:
        plt.ylabel("Accuracy (%)")
        plt.title('Overall Accuracy')
    if idx == 0:
        plt.ylabel("Accuracy (%)")
        plt.title('Head Slice Accuracy')
    if idx == 3:
        plt.ylabel("Accuracy (%)")
        plt.title('Head-Torso Overlap Accuracy')

    plt.xlabel("Overlap Portion of Head-Torso")
    plt.legend(['Model 0', 'Model 1'])
    
    sns.despine()
    plt.tight_layout()

In [6]:
N = 10000
covs = np.array([0.9, 0.9, 0.9])
m = np.shape(covs)[0]
model_0_scores = []
model_1_scores = []
model_2_scores = []

for iteration in range(50):
    for head_acc in [0.6, 0.7, 0.8, 0.9]: #head LF overall acc
        op = 0.05
        #generate data
        accs = np.array([0.75, 0.75, head_acc])
        X,Y,L,overlap_idx = generate_data(N,accs,covs,op)
       
        #train the models and score the models (I KNOW ITS UGLY)
        model_0, model_1, model_2 = train_models(X,L)
        for model, m_scores in zip([model_0, model_1, model_2],[model_0_scores, model_1_scores, model_2_scores]):
            a,b,c,d,e,f,g = eval_model(model,X,L,Y,overlap_idx)
            m_scores.append([a,b,c,d,e,f,g])
            
    
    np.save('ha_model_0_scores.npy', np.array(model_0_scores))
    np.save('ha_model_1_scores.npy', np.array(model_1_scores))
    np.save('ha_model_2_scores.npy', np.array(model_2_scores))

[Epoch 0] Loss: 4.259


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-cc6e1edb82f9>", line 16, in <module>
    model_0, model_1, model_2 = train_models(X,L)
  File "<ipython-input-3-3337d3d64302>", line 7, in train_models
    model_0.train(X, L, batch_size=1000, n_epochs=250, lr=0.1, print_every=250)
  File "/Users/vincentchen/code/metal/metal/contrib/slicing/online_dp.py", line 73, in train
    for batch, data in enumerate(train_loader):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 314, in __next__
    batch = self.collate_fn([self.dataset[i] for i in indices])
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/

KeyboardInterrupt: 

In [ ]:
# np.save(np.array(model_0_scores), 'op_model_0_scores.npy')
# np.save(np.array(model_1_scores), 'op_model_1_scores.npy')
# np.save(np.array(model_2_scores), 'op_model_2_scores.npy')

In [ ]:
plot_error(model_0_scores, model_2_scores, idx=6, iters=5, vals_list=[0.0, 0.1, 0.25])